In [1]:
#%pip install opencv-python deepface mysql-connector-python numpy scipy

In [2]:
import os
import time
import cv2
import shutil
import json
import numpy as np
from datetime import datetime
import mysql.connector
from deepface import DeepFace
from scipy.spatial.distance import cosine

# === Configuration ===

In [3]:
WATCH_FOLDER = "captured_faces"
KNOWN_FOLDER = "known_faces"
MATCH_THRESHOLD = 0.4

# === MySQL Connection ===

In [4]:
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="emotion_detection"
)
cursor = db.cursor()

# === Helper Functions ===

In [5]:
def get_face_embedding(image_path):
    try:
        embedding_obj = DeepFace.represent(img_path=image_path, model_name="Facenet", enforce_detection=False)[0]
        return embedding_obj['embedding']
    except Exception as e:
        print(f"❌ Failed to extract embedding: {e}")
        return None

def load_known_embeddings():
    cursor.execute("SELECT face_id, embedding FROM unique_face_id")
    results = cursor.fetchall()
    known = []
    for face_id, embedding_str in results:
        embedding = json.loads(embedding_str)
        known.append((face_id, np.array(embedding)))
    return known

def get_matching_face_id(image_path):
    new_embedding = get_face_embedding(image_path)
    if new_embedding is None:
        return None
    known_embeddings = load_known_embeddings()
    for known_id, known_embedding in known_embeddings:
        distance = cosine(new_embedding, known_embedding)
        if distance < MATCH_THRESHOLD:
            print(f"✅ Face matched with known UID: {known_id}")
            return known_id
    return None

def insert_user_if_not_exists(face_id, image_path):
    try:
        # First check if face exists in unique_face_id
        cursor.execute("SELECT face_id FROM unique_face_id WHERE face_id = %s", (face_id,))
        if not cursor.fetchone():
            embedding = get_face_embedding(image_path)
            if embedding is None:
                print(f"⚠️ Skipped user insertion for {face_id} due to missing embedding.")
                return False
            
            embedding_str = json.dumps(embedding)
            
            # Insert into unique_face_id first
            cursor.execute("""
                INSERT INTO unique_face_id (face_id, embedding)
                VALUES (%s, %s)
            """, (face_id, embedding_str))
            
            # Then insert into monitor_uniquefaceid
            cursor.execute("""
                INSERT INTO monitor_uniquefaceid (face_id, embedding)
                VALUES (%s, %s)
            """, (face_id, embedding_str))
            
            db.commit()
            print(f"🆕 Inserted new face ID in both tables: {face_id}")
        else:
            # Ensure the face_id exists in monitor_uniquefaceid even if it was in unique_face_id
            cursor.execute("SELECT face_id FROM monitor_uniquefaceid WHERE face_id = %s", (face_id,))
            if not cursor.fetchone():
                cursor.execute("""
                    INSERT INTO monitor_uniquefaceid (face_id, embedding)
                    SELECT face_id, embedding FROM unique_face_id WHERE face_id = %s
                """, (face_id,))
                db.commit()
                print(f"🔄 Added existing face_id to monitor_uniquefaceid: {face_id}")
        return True
    except mysql.connector.Error as err:
        print(f"❌ MySQL error in insert_user_if_not_exists: {err}")
        db.rollback()
        return False

def insert_emotion(face_id, emotion, confidence):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    try:
        # First verify the face_id exists in monitor_uniquefaceid
        cursor.execute("SELECT face_id FROM monitor_uniquefaceid WHERE face_id = %s", (face_id,))
        if not cursor.fetchone():
            print(f"⚠️ Face ID {face_id} not found in monitor_uniquefaceid, skipping emotion insert")
            return False
        
        # Now safe to insert into monitor_emotion
        cursor.execute("""
            INSERT INTO monitor_emotion (face_id, detected_emotion, confidence, timestamp)
            VALUES (%s, %s, %s, %s)
        """, (face_id, emotion, float(confidence), timestamp))
        
        # Also insert into emotions table if needed
        cursor.execute("""
            INSERT INTO emotions (face_id, detected_emotion, confidence, timestamp)
            VALUES (%s, %s, %s, %s)
        """, (face_id, emotion, float(confidence), timestamp))
        
        db.commit()
        print(f"📊 Emotion logged for {face_id}: {emotion} ({confidence:.2f}%)")
        return True
    except mysql.connector.Error as err:
        print(f"❌ MySQL error in insert_emotion: {err}")
        db.rollback()
        return False

def should_log_visit(face_id):
    try:
        cursor.execute("""
            SELECT visit_time FROM visits
            WHERE user_id = %s
            ORDER BY visit_time DESC LIMIT 1
        """, (face_id,))
        last_visit = cursor.fetchone()
        if last_visit is None:
            return True
        last_time = last_visit[0]
        diff_seconds = (datetime.now() - last_time).total_seconds()
        return diff_seconds >= 60
    except Exception as e:
        print(f"⚠️ Visit check failed: {e}")
        return False

def should_log_visit_detail(face_id):
    try:
        cursor.execute("""
            SELECT timestamp FROM visit_details
            WHERE user_id = %s
            ORDER BY timestamp DESC LIMIT 1
        """, (face_id,))
        last_detail = cursor.fetchone()
        if last_detail is None:
            return True
        last_time = last_detail[0]
        diff_seconds = (datetime.now() - last_time).total_seconds()
        return diff_seconds >= 60
    except Exception as e:
        print(f"⚠️ Visit detail check failed: {e}")
        return False

def move_image_to_archive(image_path, face_id):
    try:
        timestamp = datetime.now()
        year = str(timestamp.year)
        month = timestamp.strftime('%B')

        archive_dir = os.path.join("Process", year, month)
        os.makedirs(archive_dir, exist_ok=True)

        filename = os.path.basename(image_path)
        destination = os.path.join(archive_dir, filename)

        shutil.move(image_path, destination)
        print(f"📁 Moved {filename} to {archive_dir}")

        cursor.execute("""
            INSERT INTO archived_images (user_id, original_path, archived_path, archive_time)
            VALUES (%s, %s, %s, %s)
        """, (face_id, image_path, destination, timestamp.strftime('%Y-%m-%d %H:%M:%S')))
        db.commit()
        print(f"🗃️ Archived image logged in database for {face_id}")
        return destination
    except Exception as e:
        print(f"❌ Failed to archive image: {e}")
        return None

def insert_visitor_and_log(visitor_name, emotion, branch_id=None):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    try:
        # Only use monitor_visitor since visitor table doesn't exist
        cursor.execute("SELECT id FROM monitor_visitor WHERE name = %s", (visitor_name,))
        row = cursor.fetchone()
        
        if row:
            visitor_id = row[0]
        else:
            cursor.execute("INSERT INTO monitor_visitor (name) VALUES (%s)", (visitor_name,))
            db.commit()
            visitor_id = cursor.lastrowid

        # Insert into monitor_visitlog
        cursor.execute("""
            INSERT INTO monitor_visitlog (visitor_id, emotion, timestamp, branch_id)
            VALUES (%s, %s, %s, %s)
        """, (visitor_id, emotion, timestamp, branch_id))
        
        db.commit()
        print(f"📝 VisitLog entry for visitor {visitor_name} ({emotion})")
    except mysql.connector.Error as err:
        print(f"❌ MySQL error in insert_visitor_and_log: {err}")

def analyze_emotion(image_path, face_id, visitor_name="Unknown", branch_id=None):
    print(f"🔍 Analyzing {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Failed to read image: {image_path}")
        return
    
    try:
        results = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)
        emotion = results[0]['dominant_emotion']
        confidence = results[0]['emotion'][emotion]

        if confidence < 50:
            print(f"⚠️ Low confidence ({confidence:.2f}%) {emotion} - skipping emotion logging.")
            return

        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Insert emotion data
        insert_emotion(face_id, emotion, confidence)

        # Handle visit logging
        if should_log_visit(face_id):
            # Insert to visits table
            cursor.execute("""
                INSERT INTO visits (user_id, emotion, visit_time)
                VALUES (%s, %s, %s)
            """, (face_id, emotion, timestamp))
            
            # Insert to monitor_visit table
            cursor.execute("""
                INSERT INTO monitor_visit (face_id, visit_time)
                VALUES (%s, %s)
            """, (face_id, timestamp))
            
            db.commit()
            print(f"📝 Logged new visit for {face_id}")

            # Get the visit ID for visit details
            cursor.execute("SELECT LAST_INSERT_ID()")
            visit_id = cursor.fetchone()[0]

            if should_log_visit_detail(face_id):
                # Insert to visit_details table
                cursor.execute("""
                    INSERT INTO visit_details (user_id, timestamp, emotion, image_path)
                    VALUES (%s, %s, %s, %s)
                """, (face_id, timestamp, emotion, image_path))
                
                # Insert to monitor_visitdetail table
                cursor.execute("""
                    INSERT INTO monitor_visitdetail (visit_id, image_path)
                    VALUES (%s, %s)
                """, (visit_id, image_path))
                
                db.commit()
                print(f"📂 Logged to visit_details: {face_id}")
            else:
                print(f"⏳ Skipped visit_details (within 1-minute window)")
        else:
            print(f"⏳ Skipped visit (within 1-minute window)")

        # Handle visitor logging
        insert_visitor_and_log(visitor_name, emotion, branch_id)

        # Archive the image
        move_image_to_archive(image_path, face_id)

    except Exception as e:
        print(f"⚠️ Emotion analysis error: {e}")

def is_blurry(image_path, threshold=100.0):
    img = cv2.imread(image_path)
    if img is None:
        return True
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold

# === Monitoring Logic ===

In [6]:
def monitor_folder():
    os.makedirs(WATCH_FOLDER, exist_ok=True)
    os.makedirs(KNOWN_FOLDER, exist_ok=True)
    processed_files = set()
    print(f"📁 Monitoring {WATCH_FOLDER}...")

    while True:
        try:
            files = os.listdir(WATCH_FOLDER)
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')) and file not in processed_files:
                    path = os.path.join(WATCH_FOLDER, file)
                    print(f"\n🖼️ New image: {file}")

                    # Step 1: Try to match face
                    matched_id = get_matching_face_id(path)
                    face_id = matched_id if matched_id else os.path.splitext(file)[0]
                    is_known = bool(matched_id)

                    # Step 2: Insert into users table if not exists
                    insert_user_if_not_exists(face_id, path)

                    # Step 3: If face is new, log and copy
                    if not is_known:
                        print(f"🆕 New face detected: {face_id}")
                        
                        # Save image if not blurry and not already in known folder
                        known_face_path = os.path.join(KNOWN_FOLDER, file)
                        if not os.path.exists(known_face_path):
                            if not is_blurry(path):
                                shutil.copy(path, known_face_path)
                                print(f"📸 Saved new face: {file}")
                            else:
                                print(f"🚫 Not saving blurry image: {file}")
                        else:
                            print(f"🙅 Skipped: {file} already in known_faces.")
                    else:
                        print(f"🙅 Skipped: {file} already known.")

                    # Step 4: Analyze emotion
                    analyze_emotion(path, face_id)

                    # Step 5: Mark as processed
                    print(f"✅ Processed {file}")
                    processed_files.add(file)

            time.sleep(5)

        except Exception as e:
            print(f"❌ Error in loop: {e}")
            time.sleep(5)

# === Entry Point ===

In [ ]:
if __name__ == "__main__":
    monitor_folder()

📁 Monitoring captured_faces...

🖼️ New image: Piss-Off_user_20250523_203228.jpg


🆕 Inserted new face ID in both tables: Piss-Off_user_20250523_203228
🆕 New face detected: Piss-Off_user_20250523_203228
📸 Saved new face: Piss-Off_user_20250523_203228.jpg
🔍 Analyzing captured_faces\Piss-Off_user_20250523_203228.jpg
📊 Emotion logged for Piss-Off_user_20250523_203228: neutral (64.43%)
📝 Logged new visit for Piss-Off_user_20250523_203228
📂 Logged to visit_details: Piss-Off_user_20250523_203228
📝 VisitLog entry for visitor Unknown (neutral)
📁 Moved Piss-Off_user_20250523_203228.jpg to Process\2025\May
🗃️ Archived image logged in database for Piss-Off_user_20250523_203228
✅ Processed Piss-Off_user_20250523_203228.jpg

🖼️ New image: Piss-Off_user_20250523_203229.jpg
✅ Face matched with known UID: Piss-Off_user_20250523_203228
🙅 Skipped: Piss-Off_user_20250523_203229.jpg already known.
🔍 Analyzing captured_faces\Piss-Off_user_20250523_203229.jpg
📊 Emotion logged for Piss-Off_user_20250523_203228: neutral (98.44%)
⏳ Skipped visit (within 1-minute window)
📝 VisitLog entry for 